# Daily Energy

This notebook finds the mean energy use in the villages of Ajau, Asei, Atamali, Ayapo, and Kensio.  Ajau and Asei are grid connected while the other villages are microgrid.

Data on the number of connections is from AE Report #5, dated 26 June 2015. 

We use this to output the mean and median values for days where any power was delivered.  We alse use the connections data to determine the per household data.

We observe that the grid-connected regions consume about 5 kWh per day per household, while the microgrid areas consume less than 1 kWh per day.

## Future Work

- Look at how to calculate number of days with valid data
- Calculate uptime
- Combine multiple data files to observe longer time periods

In [1]:
import pandas as pd

def parse_file(energy_file_name):
    # read in raw CSV file for preprocessing
    energy_file = open(energy_file_name)
    # append data lines after header beginning with 'Timestamp'
    # dump lines from communications messages or empty lines and write to string
    file_string = ''
    omit_strings = ['Power Up', 'Communication Lost', 'Power Down', ',,,,,,,,,,,']
    # append data not matching omit_strings after sentinel is reached
    data_found = False
    for line in energy_file.readlines():
        if 'Timestamp' in line:
            data_found = True
        if not any(omit in line for omit in omit_strings) and data_found:
            file_string += line
    return file_string    

def create_data_frame(energy_file_name):
    # use pre-parsed string to read in data file to pandas and sort by date
    file_string = parse_file(energy_file_name)
    import io
    energy_data = pd.read_csv(io.StringIO(file_string),
                          index_col=0, 
                          parse_dates=[0], 
                          thousands=',')
    energy_data = energy_data.sort_index()
    return energy_data

def get_energy_stats(energy_data):
    energy = energy_data['kWh export']
    resampled = energy.resample('D').diff()
    return resampled.mean(), resampled.median()

In [2]:
energy_file_names = (('Ajau', 121, "../data-timeseries/Data_Logs Ajau Agustus.csv"),
                     ('Atamali', 40, "../data-timeseries/Data_Logs Atamali Agustus.csv"),
                     ('Ayapo', 103, "../data-timeseries/Data_Logs Ayapo Agustus.csv"),
                     ('Asei', 33, "../data-timeseries/Data_Logs ASEI July.csv"),
                     ('Kensio', 20, "../data-timeseries/Data_Logs Kensio Agustus.csv"))

output = {}
verbose = False
for village, connections, energy_file_name in energy_file_names: 
    energy_data = create_data_frame(energy_file_name)
    mean, median = get_energy_stats(energy_data)
    if verbose: print(village, mean, median)
    output[village] = {'mean kWh per day': mean,
                       'mean kWh per day per household': mean/connections,
                       'median kWh per day': median}

output_data = pd.DataFrame(output).T
output_data

,mean kWh per day,mean kWh per day per household,median kWh per day
Ajau,611.081857,5.050263,623.048111
Asei,186.858382,5.662375,213.208790
Atamali,15.370199,0.384255,16.787386
Ayapo,95.387185,0.926089,97.459428
Kensio,10.781886,0.539094,9.852627
